In [4]:
import warnings
warnings.simplefilter(action='ignore')

import anndata as ad
import pandas as pd
import numpy as np
import scanpy as sc
from itertools import chain
import episcanpy


import seaborn as sns
import matplotlib.pyplot as plt

AttributeError: module 'anndata._core.views' has no attribute 'AwkwardArrayView'

In [ ]:
import MultiMAP

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# load full rna and atac

In [ ]:
data_dir = "/lustre/groups/ml01/workspace/anastasia.litinetskaya/data"

In [ ]:
rna = sc.read(f'{data_dir}/neurips-cite/rna.h5ad')

In [ ]:
rna = rna.concatenate(sc.read(f'{data_dir}/neurips-multiome/rna.h5ad'), batch_key='concat_key', index_unique=None)

In [ ]:
rna_pca = rna.copy()
sc.pp.scale(rna_pca)
sc.pp.pca(rna_pca)
rna.obsm['X_pca'] = rna_pca.obsm['X_pca'].copy()

In [ ]:
del rna_pca

In [11]:
# loading hvf only because full atac requires too much RAM. Below how to load the full dataset:
# atac = sc.read('/lustre/groups/ml01/workspace/anastasia.litinetskaya/data/neurips-multiome/atac.h5ad')
atac = sc.read(f'{data_dir}/neurips-multiome/atac_hvf.h5ad')

In [12]:
MultiMAP.TFIDF_LSI(atac)

In [13]:
atac_X_lsi = atac.obsm['X_lsi'].copy()

In [14]:
atac = sc.read(f'{data_dir}/neurips-multiome/atac.h5ad')

In [15]:
# get atac_genes
atac.var.index = atac.var.index.map(lambda x: x.replace("-", ":", 1))
atac = episcanpy.tl.geneactivity(atac, gtf_file="gencode.v42.chr_patch_hapl_scaff.annotation.gtf")

In [16]:
atac.obsm['X_lsi'] = atac_X_lsi.copy()
del atac_X_lsi

In [17]:
# removing peaks that are mapped to the same gene. Duplicates will crash MultiMAP
atac = atac[:, ~atac.var.index.duplicated()]

In [18]:
atac.obs.Modality = atac.obs.Modality.astype(str) + "_atac"
atac.obs.Modality = atac.obs.Modality.astype("category")

In [19]:
adt = sc.read(f'{data_dir}/trimodal_neurips/adt.h5ad')

In [20]:
adt_pca = adt.copy()
sc.pp.scale(adt_pca)
sc.pp.pca(adt_pca)
adt.obsm['X_pca'] = adt_pca.obsm['X_pca'].copy()
del adt_pca

In [21]:
adt.obs.Modality = adt.obs.Modality.astype(str) + "_adt"
adt.obs.Modality = adt.obs.Modality.astype("category")

In [22]:
[rna.shape, atac.shape, adt.shape]

[(159510, 12059), (69249, 32129), (90261, 134)]

In [23]:
# subsample of 30k samples, 10k per dataset
adata = MultiMAP.Integration([adt[adt.obs.index.isin(adt.obs.iloc[:10000].index)], rna[rna.obs.index.isin(rna.obs.iloc[:10000].index)], atac[atac.obs.index.isin(atac.obs.iloc[:10000].index)]], ['X_pca', 'X_pca', 'X_lsi'])

KeyboardInterrupt: 

In [ ]:
sc.pl.embedding(adata, 'X_multimap', color=["Modality"])

In [24]:
rna.write_h5ad("rna.h5ad")
adt.write_h5ad("adt.h5ad")
atac.write_h5ad("atac.h5ad")

In [2]:
rna = sc.read("rna.h5ad")
atac = sc.read("atac.h5ad")
adt = sc.read("adt.h5ad")

NameError: name 'sc' is not defined

In [3]:
# trimodal integration fails with anndata concatenation error: https://github.com/Teichlab/MultiMAP/issues/10
adata = MultiMAP.Integration([rna, adt, atac], ['X_pca', 'X_pca', 'X_lsi'])

NameError: name 'MultiMAP' is not defined

In [ ]:
adata.write_h5ad("integrated.h5ad")

In [ ]:
sc.pl.embedding(adata, 'X_multimap', color=["Modality"])